# Lesson 13: 데이터 전처리

최규빈  
2023-07-25

<a href="https://colab.research.google.com/github/guebin/IP2023WIN/blob/main/posts/Day2/06_Functions, ControlFlow/2023-01-31-Lesson13.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# ref

<https://guebin.github.io/DV2022/posts/2022-10-24-8wk-12.html#fifa23-데이터분석>

# imports

In [1]:
import pandas as pd

# FIFA23 데이터분석

## FIFA23 data

`-` FIFA23라는 축구게임이 있음

`-` 게임에 실제 선수들이 나오면서 선수들의 능력치가 세밀하게 구현되어
있음

`-` 선수들 능력치에 대한 데이터셋은 캐글에 공개되어 있음 -
https://www.kaggle.com/datasets/bryanb/fifa-player-stats-database?select=FIFA23_official_data.csv

## 데이터살펴보기

`-` 일단 살펴보기

In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/guebin/DV2022/master/posts/FIFA23_official_data.csv')
df.head()

트랜스포즈하여 보는 것이 편할때도 있음

In [3]:
df.T

`-` column이름조사

In [4]:
df.keys()

-   이름에 space가 있어서 좀 거슬림

**Quiz** column이름에 space를 제거할 수 있을까?

`-` 각 column 별로 자료형조사

In [7]:
df.ID

In [9]:
df.ID.dtype

**Quiz** 각 컬럼별로 자료형을 조사할 수 있을까?

`-` 결측치조사

In [11]:
df.Club 

-   무소속인 선수도 있지 않나?

(예비학습)

In [17]:
pd.Series([1,2,None,2,3]).isna()

예비학습 끝

In [18]:
df.Club.isna().sum() # 무소속인 선수는 211명 

**Quiz** 각 컬럼별로 결측시의 수를 조사할 수 있을까?

**Quiz** 각 col별로 자료형과 결측치의 수를 조사한 표를 얻을 수 있을까?

In [7]:
# 이런식으로!!

**Quiz** 열의선택: 결측치가 10000개 이상인 열을 보고싶다면? (먼가 문제가
있는 열 같은데?)

In [8]:
df.loc[:,[df[key].isna().sum()>10000 for key in df.keys()]]

`-` .info()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17660 entries, 0 to 17659
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        17660 non-null  int64  
 1   Name                      17660 non-null  object 
 2   Age                       17660 non-null  int64  
 3   Photo                     17660 non-null  object 
 4   Nationality               17660 non-null  object 
 5   Flag                      17660 non-null  object 
 6   Overall                   17660 non-null  int64  
 7   Potential                 17660 non-null  int64  
 8   Club                      17449 non-null  object 
 9   Club Logo                 17660 non-null  object 
 10  Value                     17660 non-null  object 
 11  Wage                      17660 non-null  object 
 12  Special                   17660 non-null  int64  
 13  Preferred Foot            17660 non-null  object 
 14  Intern

## 데이터정리하기

`-` 칼럼이름변경

In [19]:
df.set_axis(pd.Index(map(lambda x: x.replace(' ','_'), df.columns)), axis=1)

`-` 결측치제거

In [20]:
df.drop(columns=['Loaned From', 'Best Overall Rating']).dropna()

`-` Height, Weight의 자료형을 float형으로 수정하기

In [21]:
df.assign(
    Height= list(map(lambda x: float(x[:-2]), df.Height)),
    Weight= list(map(lambda x: float(x[:-2]), df.Weight))
)

`-` Release Clause의 자료형을 float으로 수정하기

In [22]:
df['Release Clause']

In [23]:
_f = lambda x: float(x[1:-1])*1000 if x[-1]=='K' else float(x[1:-1])*1000000

In [24]:
_f('€157M')

In [25]:
_f('€131K')

(시도1–실패)

In [26]:
list(map(_f,df['Release Clause']))

(시도1이 실패한 이유)

In [27]:
df['Release Clause'].isna().sum() # 이 column에는 1151개의 결측치가 존재

(nan에 대한 예비학습)

In [28]:
df.loc[df['Release Clause'].isna(), 'Release Clause']

In [29]:
df.loc[18, 'Release Clause']

In [30]:
pd.isna(df.loc[18, 'Release Clause'])

In [31]:
type(df.loc[18, 'Release Clause'])

In [32]:
df.loc[18, 'Release Clause'][-1]

(시도2–성공)

In [33]:
df.rename(columns={'Release Clause':'ReleaseClause'})\
.assign(ReleaseClause = list(map(lambda x: _f(x) if pd.isna(x)==False else x , df['Release Clause'])))\
.rename(columns={'ReleaseClause':'Release Clause'})

(시도3–성공) 그냥 결측치를 제거하고 변형해도 무방..

In [34]:
df2 = df.drop(columns=['Loaned From', 'Best Overall Rating']).dropna()
df2['Release Clause'] = list(map(lambda x: _f(x) if pd.isna(x)==False else x , df2['Release Clause']))
df2

> 분석의 편의를 위하여 (1) colnames를 변경하고 (2) 결측치를 제거하고 (3)
> 몇 가지 전 처리를 추가로 진행한 뒤 df2를 만들어서 분석하는게 좋음